In [1]:
#import libraries
import numpy as np
import pandas as pd
import sys
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
eps = np.finfo(float).eps
from numpy import log2 as log
import random
from pprint import pprint

In [2]:
# Set warning message filter
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
#import the data
testingData = pd.read_csv("customer_churn_dataset-testing-master.csv")
print("Number of samples: %d" %len(testingData))
trainingData = pd.read_csv("customer_churn_dataset-training-master.csv")
print("Number of samples: %d" %len(trainingData))
finalData = pd.concat([testingData , trainingData] , ignore_index = True)
finalData.sort_values(by=["CustomerID"])

Number of samples: 64374
Number of samples: 440833


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1.0,22.0,Female,25.0,14.0,4.0,27.0,Basic,Monthly,598.00,9.0,1.0
64374,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,2.0,41.0,Female,28.0,28.0,7.0,13.0,Standard,Monthly,584.00,20.0,0.0
2,3.0,47.0,Male,27.0,10.0,2.0,29.0,Premium,Annual,757.00,21.0,0.0
64375,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
505203,449996.0,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
505204,449997.0,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
505205,449998.0,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0
505206,449999.0,31.0,Male,48.0,20.0,1.0,14.0,Premium,Quarterly,567.77,21.0,0.0


In [5]:
finalData.dropna()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1.0,22.0,Female,25.0,14.0,4.0,27.0,Basic,Monthly,598.00,9.0,1.0
1,2.0,41.0,Female,28.0,28.0,7.0,13.0,Standard,Monthly,584.00,20.0,0.0
2,3.0,47.0,Male,27.0,10.0,2.0,29.0,Premium,Annual,757.00,21.0,0.0
3,4.0,35.0,Male,9.0,12.0,5.0,17.0,Premium,Quarterly,232.00,18.0,0.0
4,5.0,53.0,Female,58.0,24.0,9.0,2.0,Standard,Annual,533.00,18.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
505202,449995.0,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
505203,449996.0,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
505204,449997.0,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
505205,449998.0,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [7]:
def train_test_split(df, test_size):
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
        
    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k = test_size)
    
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [8]:
train_df, test_df = train_test_split(finalData, 0.4)

In [9]:
def check_purity(data):
    label_column = data[: , -1]
    unique_classes = np.unique(label_column)
    
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [10]:
def get_potential_splits(data):
    potential_splits={}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        values = data[:, column_index]
        unique_values = np.unique(values)
        potential_splits[column_index] = unique_values
        
    return potential_splits

In [11]:
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]
    data_below = data[split_column_values == split_value]
    data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [12]:
def calculate_mse(data): 
    actual_values = data[:, - 1]
    if len(actual_values) == 0:
        mse = 0
    else:
        prediction = np.mean(actual_values)
        mse = np.mean((actual_values - prediction)**2)
        
    return mse

In [13]:
def calculate_entropy(data):
    label_column =data[:, -1]
    _, counts = np.unique(label_column, return_counts = True)
    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    
    return entropy

In [14]:
def calculate_overall_metric(data_below, data_above, metric_function):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below)/n
    p_data_above = len(data_above)/n
    overall_metric = (p_data_below * metric_function(data_below) + p_data_above * metric_function(data_above))
    
    return overall_metric

In [15]:
def determine_best_split(data, potential_splits):
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column = column_index, split_value = value)
            current_overall_metric = calculate_overall_metric(data_below, data_above, metric_function = calculate_entropy)
            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
                
    return best_split_column, best_split_value

In [16]:
def findEntropy(df):
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value] / len(df[Class])
        entropy += - fraction * np.log2(fraction)
        
    return entropy

In [17]:
def findEntropyAttribute(df, attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables =df[attribute].unique()
    entropy2 = 0
    min_entropy = -999
    min_var_name = ""
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] ==variable][df[Class] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num/(den + eps)
            entropy += -fraction * log(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
        if(entropy2 >= min_entropy):
            min_entropy = entropy2
            min_var_name = variable
            
    return abs(entropy2), min_var_name

In [18]:
def findSplitInfo(df, attribute):
    variables = df[attribute].unique()
    if(len(variables) == 1):
        return 1;
    split_info = 0
    for variable in variables:
        d = len(df[df[attribute] == variable]) / len(df) 
        split_info += d * log(d)
    
    return abs(split_info)

In [19]:
def infoGain(df, ratio = False):
    df = pd.DataFrame(df, columns = [COLUMN_HEADERS])
    IG = []
    var_names = []
    for key in df.keys()[: -1]:
        val, var_name = findEntropyAttribute(df, key)
        ig = findEntropy(df) - val
        if(ratio):
            split_info = findSplitInfo(df, key)
            gain_ratio = ig/split_info
            IG.append(gain_ratio)
            var_names.append(var_name)
        else:
            IG.append(ig)
            var_names.append(var_name)
    best_ig_index = np.argmax(IG)
    
    return best_ig_index, var_names[best_ig_index]

In [20]:
def giniImpurity2(valueCounts):
    n = valueCounts.sum()
    p_sum = 0
    for key in valueCounts.keys():
        p_sum = p_sum + (valueCounts[key] / n) * (valueCounts[key] / n)
        gini = 1 - p_sum
        
    return gini

In [21]:
def giniSplitAtt2(df, attName):
    attValues = df[attName].value_counts()
    gini_A = 0 
    min_impurity = 999
    min_impurtiy_variable = "";
    for key in attValues.keys():
        dfKey = df[df.keys()[-1]][df[attName] == key].value_counts()
        numOfKey = attValues[key] = df.shape[0]
        impurity = ((numOfKey / n)*giniImpurity2(dfKey))
        if(impurity <= min_impurity):
            min_impurity = impurity 
            min_impurtiy_variable = key
        gini_A = gini_A + impurity
    
    return gini_A, min_impurtiy_variable

In [22]:
def giniIndex2(df, attributeNames):
    df = pd.DataFrame(df, columns = COLUMN_HEADERS)
    giniAttribute = {}
    minValue = sys.maxsize
    counter = 0
    for key in attributeNames:
        giniAttribute[key] = [*giniSplitAtt2(df, key)]
        if giniAttribute[key][0] < minValue:
            minValue = giniAttribute[key][0]
            selectedAttribute = counter
            selectedVariable = giniAttribute[key][1]
        counter += 1
        
    return selectedAttribute, selectedVariable

In [23]:
def decision_tree_algorithm(df, counter = 0, min_samples = 2, max_depth = 5, model = 'infoGain'):
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df
    if(check_purity(data)) or(len(data) < min_samples) or(counter ==max_depth):
        classification = classify_data(data)
        return classification
    else: 
        counter += 1
        if(model == 'infoGain'):
            split_column, split_value =infoGain(data)
        elif(model == 'gainRatio'):
            split_column, split_value = infoGain(data, True)
        elif(model == 'gini'):
            split_column,split_value = giniIndex2(data, COLUMN_HEADERS[:-1])
        data_below, data_above = split_data(data, split_column, split_value)
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} = {}".format(feature_name, split_value)
        sub_tree = {question: []}
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, model)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, model)
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree


defining a random forest model

In [24]:
class RandomForest:
  def __init__(self, n_trees = 10, max_depth = 10, min_samples_split = 2, n_features=None):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.n_features = n_feature
    self.trees = []

  def _bootstrap_sample(self, X, y):
    n_samples = X.shape[0]
    idx = np.random.choice(n_samples, n_samples, replace = True)
    return X[idx], y[idx]

  def fit(self, X, y):
    self.trees = []
    for _ in range(self.n_trees):
      tree = decision_tree_algorithm(df, counter = 0, min_samples = 2, max_depth = 5, model = 'infoGain')
      X_sample, y_sample = self.bootstrap_sample(X,y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)

  def _most_common_label (self, y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

  def predict(self, X):
    predictions = np.array([tree.predict(X) for tree in self.trees])
    tree_preds = np.swapaxes(predictions, 0,1)
    predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
    return predictions

In [25]:
x = train_df.drop(columns=["churn"]).values
y = train_df["churn"].values

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=0)

random_forest = RandomForest(n_trees = 10, max_depth = 3, min_samples_split = 2, n_features=None)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

KeyError: "['churn'] not found in axis"

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)